## 04. 텍스트 분류 실습 - 20 뉴스 그룹 분류

* 텍스트 분류 : 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤 이 학습 모델을 이용해 다른 문서의 분류를 예측하는 것
  - 텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬 형태가 됨
  - 희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘은 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등

1. 텍스트를 정규화한 뒤 피처 벡터화를 적용
2. 적합한 머신러닝 알고리즘을 적용해 분류를 학습/예측/평가

  - 카운트 기반과 TF-IDF 기반의 벡터화를 차례로 적용해 예측 성능을 비교
  - 피처 벡터화를 위한 파라미터와 그리드서치기반의 하이퍼 파라미터 튜닝
  - 사이킷런의 파이프라인 객체를 통한 피처 벡터화 파라미터와 그리드서치 기반의 하이퍼 파라미터 튜닝을 한꺼번에 수행하는 방법

### 텍스트 정규화

In [4]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset = 'all', random_state = 156)

In [5]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [6]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [7]:
#개별 데이터가 텍스트로 어떻게 구성돼 있는지 데이터를 한 개만 추출해 값 확인 
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [8]:
from sklearn.datasets import fetch_20newsgroups
# subset = 'train'으로 학습용 데이터만 추출, remove=('headers', 'footers', 'quotes') 로 내용만 추출
train_news = fetch_20newsgroups(subset = 'train', remove=('headers', 'footers', 'quotes'), random_state = 156)

X_train = train_news.data
y_train = train_news.target

#subset='test'으로 테스트 데이터만 추출, remove=('headers', 'footers', 'quotes') 로 내용만 추출

test_news = fetch_20newsgroups(subset = 'train', remove=('headers', 'footers', 'quotes'), random_state = 156)

X_test = train_news.data
y_test = train_news.target
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 11314


### 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

학습 데이터: 11314개의 뉴스그룹 문서, 리스트형태 / 테스트 데이터: 7532개의 문서, 리스트 형태

CountVectorizer : 텍스트를 피처 벡터화 
- 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 테스트 데이터를 변환해야 함!! 
  - 그래야만 학습시 설정된 피처 개수와 테스트 데이터를 변환할 피처 개수가 같아짐
- 테스트 데이터의 피처 벡터화시 fit_transform() 사용하면 안됨! 
  - 학습 시 사용된 피처 개수와 예측 시 사용할 피처 개수 달라짐

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

#Count Vectorization으로 피처 벡터화 변환 수행.
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

#학습 데이터로 fit()된 CountVectorizer를 이용해 체스트 데이터를 피터 벡처화 변환 수행.
X_test_cnt_vect= cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 countVectorizer shape:', X_train_cnt_vect.shape)

학습 데이터 텍스트의 countVectorizer shape: (11314, 101631)


> 11314개의 문서에 피처, 즉 단어가 101631개 만들어짐

피처 벡터화된 데이터에 로지스틱 회귀를 적용해 뉴스그룹에 대한 분류 예측

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#LogisticRegression을 이용해 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred= lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

C:\Users\chltp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CountVectorized Logistic Regression의 예측 정확도는 0.964


Count 기반에서 TF-IDF 기반으로 벡터화를 변경해 예측 모델 수행

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

#TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환.
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

#LogisticRegression을 이용해 학습/예측/평가 수행

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred= lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.907


> TF-IDF가 단순 카운트 기반보다 훨씬 높은 예측 정확도 제공

일반적으로 문서 내에 텍스트가 많고, 많은 문서를 가지는 텍스트 분석에서 카운트 벡터화 보다는 TF-IDF 벡터화가 좋은 예측 결과를 도출

**텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 방법**
1. 최적의 ML 알고리즘 선택
2. 최상의 피처 전처리 수행

텍스트 정규화나 Count/TF-IDF 기반 피처 벡터화를 어떻게 효과적으로 적용했는지가 텍스트 기반의 머신러닝 성능에 큰 영향을 미침

TF-IDF 다양한 파라미터 적용

In [ ]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidt_vect = tfidf_vect.transform(X_train)
X_test_tfidt_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidt_vect, y_train)
pred = lr_clf.predict(X_test_tfidt_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))
     

GridSearchCV 로지스틱 회귀의 하이퍼 파라미터 최적화 수행

In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 수행. CV는 3폴드 세트로 설정
params = { 'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidt_vect, y_train)
print('Logistic Regression best C parameter :', grid_cv_lr.best_params_)

# 최적 C값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidt_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

> 로지스틱 회귀의 C=10 일때 교차 검증 테스트 세트에서 가장 좋은 예측 성능, 테스트 데이터 세트에 적용해 성능 수치 향상

### 사이킷런 파이프라인 사용 및 GridSearchCV 와의 결합

피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성을 한 번에 진행
- 데이터 전처리와 머신런이 학습 과정을 통일된 API 기반에서 처리할 수 있어 더 직관적인 ML 모델 코드 생성 가능
- 대용량 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 스트림 기반에서 바로 머신러닝 알고리즘의 데이터로 입력할 수 있어 수행 시간 절약

텍스트 기반의 피처 벡터화 뿐만 아니라, 모든 데이터 전처리 작업과 estimator 결합 가능

In [ ]:
from sklearn.pipeline import Pipeline

# TfidVectorizer 객체를 tfid_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

# 별도의 TfidVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요없음
# pipeline의 fit과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))

GridSearchCV에 Pipeline을 입력하면서 TfidVectorizer 파라미터와 Logistic Regression의 하이퍼 파라미터를 함께 최적화

In [ ]:
from sklearn.model_selection import Pipeline

pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(__) 2개를 연달아 붙여 GridSearchCV에 사용될 파라미터/하이퍼파라미터 이름과 값을 설정
params = {'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],
         'tfidf_vect__max_df': [100,300,700],
         'lr_clf__C':[1,5,10]}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}.format(accuracy_score(y_test,pred))')